In [75]:
import random
import copy
import imageio
import time
import numpy as np

config = {
    "height": 10,
    "width": 25,
    "rounds_to_play": 60,
    "scale_up_by": 14,
    "output_format": ".gif",
    "frames_per_second": 8
}

In [76]:
def create_canvas(height, width):
    '''creates the canvas on which game of life will be played'''
    game = []
    
    for line in range(height):
        filled_line = []
        filled_line_uint8 = []
        for position in range(width):
            filled_line.append(random.randint(0, 1))
        game.append(filled_line)
    
    return game

In [79]:
def cor(pos):
    '''corrects position within a line in case it is out of range'''
    if pos < 0:
        return 0
    if pos > width:
        return width
    return pos

def check_line(line):
    '''returns IndexError if a line is -1. reason: it doesn't exist, \
    but python will evaluate the last line (line[-1])'''
    if line == -1:
        raise IndexError 
    return line


def check_position(game, line, pos):
    '''returns count of life around a given position (line and pos) \
    within a specified game of life (game)'''
    
    surrounding = sum(game[line][cor(pos-1):cor(pos+2)]) - game[line][pos]
    
    if not line == 0:
        try:
            surrounding += sum(game[check_line(line-1)][cor(pos-1):cor(pos+2)])
        except IndexError:
            pass
        
    try:
        surrounding += sum(game[line+1][cor(pos-1):cor(pos+2)])
    except IndexError:
        pass
    
    return surrounding


def alive_or_dead(this_round, line, pos):
    '''evaluates whether a cell should be alive or dead in the next round'''
    life_around = check_position(this_round, line, pos)
    
    if life_around == 3:
        return 1
    
    if life_around == 2:
        return this_round[line][pos]
    
    return 0

In [83]:
def scale_for_animation(frame, times_big):
    '''scales up single round and converts it to uint8 \
    so it can be visualized with imageio'''
    converted_frame = []
    for line in frame:
        filled_line_uint8 = [255 if x == 1 else 0 for x in line]
        converted_frame.append(filled_line_uint8) 
    return np.kron(converted_frame, np.ones((times_big,times_big))).astype(np.uint8)


def create_visual(frames, fps, output_format):
    '''writes and saves played game of life. uses imageio; \
    output can be ".gif" or ".mp4"'''
    name = str(int(time.time())) + output_format
    writer = imageio.get_writer(name, fps=fps)
    
    for frame in frames:
        writer.append_data(frame)

    writer.close()
    
    print("Done. Saved game as {}; bye.".format(name))

In [80]:
def life_generator():
    '''generator that plays the game'''
    
    this_round = create_canvas(height, width)
    yield this_round
    
    while True:
        
        next_round = []
        
        for line in range(0, height):
            new_line = []
            
            for pos in range(0, width):
                new_pos = alive_or_dead(this_round, line, pos)
                new_line.append(new_pos)
               
            next_round.append(new_line)
        
        this_round = copy.deepcopy(next_round)
        yield next_round

In [81]:
def play_conway(game_rounds, round_count, rounds_to_play, scale_up_by):
    print("Let's play:")
    print("Starting Game of Life with {} cells and {} rounds...".format(height * width, rounds_to_play))
    
    for game_round in game_rounds: 
        scaled_canvas = scale_for_animation(game_round, scale_up_by)
        visual.append(scaled_canvas)
    
        if round_count >= rounds_to_play:
            print("Finished game after {} rounds.")
            print("Proceeding to creating image".format(round_count))
            create_visual(visual, fps, output_format)
            break
        
        round_count += 1

In [82]:
rounds_to_play = config["rounds_to_play"]
height = config["height"]
width = config["width"]
output_format = config["output_format"]
scale_up_by = config["scale_up_by"]
fps = config["frames_per_second"]

game_rounds = life_generator()
round_count = 1
visual = []

play_conway(game_rounds, round_count, rounds_to_play, scale_up_by)

Let's play:
Starting Game of Life with 250 cells and 60 rounds...
Finished game after {} rounds.
Proceeding to creating image
Done. Saved game as 1544169519.gif; bye.
